In [1]:
import os
import glob
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from h2o.automl import H2OAutoML

## Loading the dataset ##

In [2]:
# Folder containing CSV files
DATA_DIR = "MachineLearningCSV"

# Search for all CSV files in the folder
files = sorted(glob.glob(os.path.join(DATA_DIR, "*.csv")))
if len(files) == 0:
    raise FileNotFoundError(f"No CSV files found in {DATA_DIR}. Check the path or files.")

print(f"CSV files found ({len(files)}):\n", "\n".join(files))

# Reading and concatenation
start = time.time()
df_list = []
for f in files:
    print("Reading:", f)
    # Read using ';' separator
    df_part = pd.read_csv(f, sep=';', low_memory=False)
    df_list.append(df_part)

# Concatenate all files into a single DataFrame
df = pd.concat(df_list, ignore_index=True)
print(f"Concatenation completed. Shape: {df.shape} (time: {time.time()-start:.2f}s)")

# DataFrame preview
display(df.head())

CSV files found (8):
 MachineLearningCSV\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
MachineLearningCSV\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
MachineLearningCSV\Friday-WorkingHours-Morning.pcap_ISCX.csv
MachineLearningCSV\Monday-WorkingHours.pcap_ISCX.csv
MachineLearningCSV\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
MachineLearningCSV\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
MachineLearningCSV\Tuesday-WorkingHours.pcap_ISCX.csv
MachineLearningCSV\Wednesday-workingHours.pcap_ISCX.csv
Reading: MachineLearningCSV\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Reading: MachineLearningCSV\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Reading: MachineLearningCSV\Friday-WorkingHours-Morning.pcap_ISCX.csv
Reading: MachineLearningCSV\Monday-WorkingHours.pcap_ISCX.csv
Reading: MachineLearningCSV\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Reading: MachineLearningCSV\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


## DataFrame info ##

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

## The dataset is too large for H2O on 8GB RAM; consider moving to the cloud or skipping this step ##

In [4]:
# Frequencies in the original dataset
print("Original distribution:")
print(df[' Label'].value_counts(normalize=True))

Original distribution:
 Label
BENIGN                        0.803004
DoS Hulk                      0.081630
PortScan                      0.056144
DDoS                          0.045227
DoS GoldenEye                 0.003636
FTP-Patator                   0.002804
SSH-Patator                   0.002083
DoS slowloris                 0.002048
DoS Slowhttptest              0.001943
Bot                           0.000695
Web Attack � Brute Force      0.000532
Web Attack � XSS              0.000230
Infiltration                  0.000013
Web Attack � Sql Injection    0.000007
Heartbleed                    0.000004
Name: proportion, dtype: float64


## The dataset is highly imbalanced; apply the following step to balance it ##

In [5]:
import pandas as pd

# Name of your target column (note the leading space!)
target = ' Label'

df_sample = (
    df
    .groupby(target, group_keys=False)
    .apply(
        lambda x: x.sample(
            frac=0.20 if x.name == "BENIGN" else 0.80,
            random_state=1234
        )
    )
)

C:\Users\HP\AppData\Local\Temp\ipykernel_13772\1082550615.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


## Verification of changes ##

In [6]:
print("Original dataset size:", len(df)) # verify original dataset
print("Sample size:", len(df_sample)) # verify reduced dataset

Original dataset size: 2830743
Sample size: 900737


In [7]:
# Frequencies in the original dataset
print("Original distribution:")
print(df[' Label'].value_counts(normalize=True))

# Frequencies in the sample
print("\nDistribution in the sample:")
print(df_sample[' Label'].value_counts(normalize=True))

Original distribution:
 Label
BENIGN                        0.803004
DoS Hulk                      0.081630
PortScan                      0.056144
DDoS                          0.045227
DoS GoldenEye                 0.003636
FTP-Patator                   0.002804
SSH-Patator                   0.002083
DoS slowloris                 0.002048
DoS Slowhttptest              0.001943
Bot                           0.000695
Web Attack � Brute Force      0.000532
Web Attack � XSS              0.000230
Infiltration                  0.000013
Web Attack � Sql Injection    0.000007
Heartbleed                    0.000004
Name: proportion, dtype: float64

Distribution in the sample:
 Label
BENIGN                        0.504719
DoS Hulk                      0.205230
PortScan                      0.141156
DDoS                          0.113709
DoS GoldenEye                 0.009141
FTP-Patator                   0.007050
SSH-Patator                   0.005238
DoS slowloris                 0.005148
DoS 

## Non-null values of the reduced dataframe ##

In [8]:
df_sample.notna().sum()

 Destination Port              900737
 Flow Duration                 900737
 Total Fwd Packets             900737
 Total Backward Packets        900737
Total Length of Fwd Packets    900737
                                ...  
Idle Mean                      900737
 Idle Std                      900737
 Idle Max                      900737
 Idle Min                      900737
 Label                         900737
Length: 79, dtype: int64

## Handling missing values ##

In [9]:
df_sample.isna().sum()[df_sample.isna().sum() > 0] # Shows only columns with missing values

Flow Bytes/s    845
dtype: int64

In [10]:
(df_sample.isna().sum() / len(df_sample) * 100).round(4) # See percentage of missing values

 Destination Port              0.0
 Flow Duration                 0.0
 Total Fwd Packets             0.0
 Total Backward Packets        0.0
Total Length of Fwd Packets    0.0
                              ... 
Idle Mean                      0.0
 Idle Std                      0.0
 Idle Max                      0.0
 Idle Min                      0.0
 Label                         0.0
Length: 79, dtype: float64

In [11]:
df_sample = df_sample.dropna(subset=['Flow Bytes/s']) # Drop rows with missing Flow Bytes/s (recommended here)

In [12]:
df_sample.notna().sum() # verification once more to see if everything is OK

 Destination Port              899892
 Flow Duration                 899892
 Total Fwd Packets             899892
 Total Backward Packets        899892
Total Length of Fwd Packets    899892
                                ...  
Idle Mean                      899892
 Idle Std                      899892
 Idle Max                      899892
 Idle Min                      899892
 Label                         899892
Length: 79, dtype: int64

In [13]:
df_sample.isna().sum()[df_sample.isna().sum() > 0] # Shows only columns with missing values

Series([], dtype: int64)

In [14]:
# Clean column names (recommended before export)
# Before saving, remove leading/trailing spaces from column names:
df_sample.columns = df_sample.columns.str.strip()

In [15]:
df_sample.to_csv("dataset_sample_10pct.csv", index=False) # save the reduced dataset

In [16]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 899892 entries, 1108263 to 1602567
Data columns (total 79 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Destination Port             899892 non-null  int64  
 1   Flow Duration                899892 non-null  int64  
 2   Total Fwd Packets            899892 non-null  int64  
 3   Total Backward Packets       899892 non-null  int64  
 4   Total Length of Fwd Packets  899892 non-null  int64  
 5   Total Length of Bwd Packets  899892 non-null  int64  
 6   Fwd Packet Length Max        899892 non-null  int64  
 7   Fwd Packet Length Min        899892 non-null  int64  
 8   Fwd Packet Length Mean       899892 non-null  float64
 9   Fwd Packet Length Std        899892 non-null  float64
 10  Bwd Packet Length Max        899892 non-null  int64  
 11  Bwd Packet Length Min        899892 non-null  int64  
 12  Bwd Packet Length Mean       899892 non-null  float64
 1

## Memory optimization when the dataframe is too large (optional) ##

In [17]:
# This code reduces the dataframe size for optional manipulation
df_optimized = df.copy()

for col in df_optimized.select_dtypes(include=['int64']).columns:
    df_optimized[col] = pd.to_numeric(df_optimized[col], downcast='integer')

for col in df_optimized.select_dtypes(include=['float64']).columns:
    df_optimized[col] = pd.to_numeric(df_optimized[col], downcast='float')

print("Memory before:", df.memory_usage(deep=True).sum() / 1e6, "MB")
print("Memory after:", df_optimized.memory_usage(deep=True).sum() / 1e6, "MB")

Memory before: 1923.001689 MB
Memory after: 1039.809873 MB


## Data preparation ##

In [18]:
# Initialize H2O
import h2o
h2o.init(max_mem_size="5G")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 24.0.1+9-30, mixed mode, sharing)
  Starting server from C:\Users\HP\Downloads\test-H2O\.venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\HP\AppData\Local\Temp\tmp22ao24p0
  JVM stdout: C:\Users\HP\AppData\Local\Temp\tmp22ao24p0\h2o_HP_started_from_python.out
  JVM stderr: C:\Users\HP\AppData\Local\Temp\tmp22ao24p0\h2o_HP_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,13 secs
H2O_cluster_timezone:,Africa/Lagos
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.8
H2O_cluster_version_age:,2 months and 19 days
H2O_cluster_name:,H2O_from_python_HP_nnrqgz
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4.983 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [19]:
# Convert the pandas DataFrame to an H2OFrame object
# Clean the column names before conversion
df_sample.columns = df_sample.columns.str.strip()
hf = h2o.H2OFrame(df_sample)
# Ensure the H2O object has proper column names
hf.columns = [col.strip() for col in hf.columns]

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [20]:
# Just like in pandas, we can view the statistical summary of an H2O dataframe
hf.describe()

Rows:899892
Cols:79

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
type,int,int,int,int,int,int,int,int,real,real,int,int,real,real,real,real,real,real,int,int,int,real,real,int,int,int,real,real,int,int,int,int,int,int,int,int,real,real,int,int,real,real,real,int,int,int,int,int,int,int,int,int,real,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,real,real,int,int,enum
mins,0.0,-2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-193000000.0,-2000000.0,-2.0,0.0,-2.0,-14.0,0.0,0.0,0.0,0.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1073741320.0,-1073741320.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1073741320.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,-536870660.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
mean,6028.101061016215,20206669.814518858,6.99743191405191,7.2124866095042535,419.8752161370477,10926.199628399856,173.2515901908229,12.579129495539455,45.81470620880529,60.40928620839372,1591.320725153686,26.544655358643038,526.482707397285,657.6044390367675,1049046.9541624442,79830.5397009855,1888283.7088046682,4954502.659336586,16332319.633001514,184938.66665777718,19931965.20822388,3683285.0448970567,6331382.343829346,16212311.63883445,838341.3385828522,9488572.196684716,1839667.4891442065,2233079.4357172083,6104480.896855398,735180.2346881632,0.033638481062171904,0.0,6.556342316633552e-05,0.0,-2225.8471372120202,-1107.1843454547911,71917.44729200967,7918.453114523618,10.72128210940868,1643.2004996155083,267.1236778521225,518.4665336919038,1048221.4619783087,0.06168295751045681,0.033638481062171904,0.00015668546892293742,0.3591364297048979,0.35901974903655104,0.0632164748658728,6.556342316633552e-05,0.00015668546892293742,0.6577944908944626,295.13445418826285,45.81470620880529,526.4827073972928,-2225.8471372120202,0.0,0.0,0.0,0.0,0.0,0.0,6.99743191405191,419.8752161370477,7.2124866095042535,10926.199916212168,7149.802191818576,1367.112673520821,3.426521182541903,-756.4286658843455,89176.07069689197,33165.37724733788,139268.8954118937,69285.00732532355,15253719.729694283,871525.9477473663,15916900.14413286,14608160.588173915,
maxs,65534.0,119999997.0,219759.0,291922.0,2866110.0,655453030.0,24820.0,2065.0,5940.857143,7049.469004,17376.0,2042.0,5800.5,8194.660487,2071000000.0,4000000.0,120000000.0,84700000.0,120000000.0,120000000.0,120000000.0,120000000.0,84602929.2769822,120000000.0,120000000.0,120000000.0,120000000.0,84418013.7826341,120000000.0,120000000.0,1.0,0.0,1.0,0.0,4480016.0,5838440.0,3000000.0,2000000.0,1448.0,24820.0,2920.0,4731.522394,22400000.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,90.0,3337.142857,5940.857143,5800.5,4480016.0,0.0,0.0,0.0,0.0,0.0,0.0,219759.0,2866110.0,291922.0,655453030.0,65535.0,65535.0,213557.0,93.0

In [21]:
# Number of rows in the dataframe
hf.nrows

899892

In [22]:
# Remove leading/trailing spaces since they can prevent execution
hf.columns = [col.strip() for col in hf.columns]

# Verify
print(hf.columns)

['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count

In [23]:
# Independent variables (features) and target variable
# Split train/test with hf
train, test = hf.split_frame(ratios=[0.8], seed=1234)
y = 'Label'       # the column you want to predict
x = list(train.columns)  # all columns as a list
# Safe removal of the target column (won't raise if absent)
if y not in x:
    print(f"Error: the target column {y!r} is not in the train columns.")
    print('Available columns :', x[:50])
    x = [c for c in x if c != y]  # safely remove if present
else:
    x.remove(y)

In [24]:
print(train.columns) # shows the column names

['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count

## Running H2O ##

In [25]:
# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()
df_sample.columns = df_sample.columns.str.strip()

# Verify
print(df_sample.columns[-5:])  # 'Label' should appear without spaces

# Then, build the frequency table
y = 'Label'
print(df[y].value_counts(normalize=True))

Index(['Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'Label'], dtype='object')
Label
BENIGN                        0.803004
DoS Hulk                      0.081630
PortScan                      0.056144
DDoS                          0.045227
DoS GoldenEye                 0.003636
FTP-Patator                   0.002804
SSH-Patator                   0.002083
DoS slowloris                 0.002048
DoS Slowhttptest              0.001943
Bot                           0.000695
Web Attack � Brute Force      0.000532
Web Attack � XSS              0.000230
Infiltration                  0.000013
Web Attack � Sql Injection    0.000007
Heartbleed                    0.000004
Name: proportion, dtype: float64


In [26]:
# Check your x and y before training
print("Target column:", y)
print("Number of features:", len(x))
# This ensures your target column is correct and everything is configured properly.

Target column: Label
Number of features: 78


In [27]:
# Create an H2O AutoML model
aml = H2OAutoML(
    max_runtime_secs = 1800,      # AutoML stops after 30 minutes
    balance_classes = True,      # automatic class rebalancing for minority/majority classes
    stopping_metric = 'logloss', # stopping criterion based on logloss (suitable for multiclass)
    project_name = 'Final',      # AutoML project name
    seed = 123                   # for reproducibility
)
# Train the model
%time aml.train(x=x, y=y, training_frame= train)

AutoML progress: |
22:07:30.843: AutoML: XGBoost is not available; skipping it.
22:07:32.208: _train param, Dropping bad and constant columns: [Bwd Avg Packets/Bulk, Fwd Avg Bulk Rate, Bwd PSH Flags, Bwd URG Flags, Bwd Avg Bytes/Bulk, Fwd Avg Bytes/Bulk, Fwd Avg Packets/Bulk, Bwd Avg Bulk Rate]

████████████████████████████
22:20:17.407: _train param, Dropping bad and constant columns: [Bwd Avg Packets/Bulk, Fwd Avg Bulk Rate, Bwd PSH Flags, Bwd URG Flags, Bwd Avg Bytes/Bulk, Fwd Avg Bytes/Bulk, Fwd Avg Packets/Bulk, Bwd Avg Bulk Rate]

███████████████████████████
22:33:55.840: GBM_1_AutoML_1_20251227_220728 [GBM def_5] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_1_AutoML_1_20251227_220728.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (122,9 KB per tree x 10000 > 977,9 MB) - try decreasing ntrees and/or max_depth or increasing min_rows!

22:33:55.940: _train param, Dropping bad and

Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_1_AutoML_1_20251227_220728


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    1                  15                          27576                  3            20           17.5333       4             490           140.333

ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.16203177647269137
RMSE: 0.4025317086549721
LogLoss: 0.5565703272998439
Mean Per-Class Error: 0.22681251459170776
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
BENIGN    Bot    DDoS    DoS GoldenEye    DoS Hulk    DoS Slowhttptest    DoS slowloris    FTP-Patator    Heartbleed    Infiltration    PortScan    SSH-Patator    Web Attack <0xEFBFBD> Brute Force    Web Attack <0xEFBFBD> Sql Injection    Web Attack <0xEFBFBD> XSS    Error        Rate
--------  -----  ------  ---------------  ----------  ------------------  ---------------  -------------  ------------  --------------  ----------  -------------  -----------------------------------  -------------------------------------  ---------------------------  -----------  -----------------
61757     26     9       4                146         1                   6                1              0             0               22          0              10                                   0                                      2                            0.00366224   227 / 61 984
35140     26751  0       0                0           0                   0                0              0             0               0           0              0                                    0                                      0                            0.567772     35 140 / 61 891
30        0      61607   0                15          0                   0                0              0             0               5           0              0                                    0                                      2                            0.000843348  52 / 61 659
56        0      0       61054            478         10                  0                0              0             0               0           0              0                                    0                                      0                            0.00883146   544 / 61 598
26        0      0       0                62040       0                   0                0              0             0               0           0              1                                    0                                      0                            0.000435014  27 / 62 067
55        0      0       26               0           61421               244              0              0             0               0           0              0                                    0                                      0                            0.0052635    325 / 61 746
90        0      0       58               0           52                  61941            0              0             0               0           0              0                                    0                                      0                            0.00321849   200 / 62 141
19        0      0       0                0           0                   0                61481          0             0               0           0              43 

In [28]:
# 1. Show the AutoML leaderboard of top models
lb = aml.leaderboard
print("=== Top 10 AutoML models ===")
print(lb.head(rows=10))

# 2. Show the best model (leader)
leader_model = aml.leader
print("\n=== Best model found ===")
print(f"Model ID: {leader_model.model_id}")

# 3. Evaluate the model on the test set
perf = leader_model.model_performance(test)
print("\n=== Evaluation on the test set ===")
print(perf)

# 4. Print useful metrics (multinomial)
print("\nLogloss :", perf.logloss())
print("MSE :", perf.mse())
print("RMSE :", perf.rmse())
print("Hit Ratio @1 (Accuracy) :", perf.hit_ratio_table()[0][1])
print("\nConfusion matrix :")
print(perf.confusion_matrix())

# 5. Save the best model locally
model_path = h2o.save_model(model=leader_model, path="models", force=True)
print(f"\nModel saved at: {model_path}")

=== Top 10 AutoML models ===
model_id                          mean_per_class_error    logloss      rmse         mse
DRF_1_AutoML_1_20251227_220728                0.222402  0.0443963  0.071496  0.00511167
GBM_2_AutoML_1_20251227_220728                0.705868  0.682838   0.472792  0.223532
GBM_3_AutoML_1_20251227_220728                0.742374  0.725026   0.490297  0.240391
GLM_1_AutoML_1_20251227_220728                0.805516  0.981896   0.569387  0.324202
[4 rows x 5 columns]


=== Best model found ===
Model ID: DRF_1_AutoML_1_20251227_220728

=== Evaluation on the test set ===
ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.0054411091832452544
RMSE: 0.07376387451351274
LogLoss: 0.05280133690848892
Mean Per-Class Error: 0.2737306970750483
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parame